## 属性描述符相关魔法函数

### 1. 描述符(Descriptor)
- **描述符**是实现了特定协议的类。  
- **描述符功能**：是对多个属性运用相同存取逻辑的一种方式。  
- **描述符协议**：包括`__get__`、`__set__`和`__delete__`方法，通常实现部分协议。  
> 大多数描述符只实现了`__get__`和`__set__`方法，而`property`类实现了完整的描述符协议。

- 只要实现了`__get__`、`__set__`和`__delete__`中的一个就可以认为是描述符
- **Non-data Descriptor**:只实现`__get__`方法的对象是**非数据描述符**，在初始化之后，它们只能被读取
- **Data Descriptor**: 同时实现了`__get__`和`__set__`的对象是**数据描述符**，这种属性是可读可写的

描述符对象一般是作为其它类对象的属性而存在。  
内部定义了三个方法用来实现属性的查找(get)、设置(set)和删除(delete)行为。
- `__get__(self, obj, objtype=None)`: 定义当尝试获取描述符的值时的行为
- `__set__(self, obj, value)`: 定义当尝试设置或者改变描述符的值时的行为
- `__delete__(self, obj)`: 定义描述符的值被删除时的行为

### 2. 非数据描述符
> `non-data descriptor`，就是只实现了`__get__`方法的描述符。

In [1]:
class BirthdayField(object):
    
    def __init__(self, value):
        self.value = value
    
    def __get__(self, obj, objtype=None):
        # 这里obj会是引用到BirthdayField的类
        # objtype是引用这个属性描述符的类
        print("__get__:")
        print("\t self:{}".format(self))
        print("\t obj:{}".format(obj))
        print("\t Object Type:{}".format(objtype))
        return self.value

In [2]:
class People:
    
    birth = BirthdayField("2000年1月1日")

In [3]:
print(People.birth)

__get__:
	 self:<__main__.BirthdayField object at 0x1056e0650>
	 obj:None
	 Object Type:<class '__main__.People'>
2000年1月1日


In [4]:
# 实例化People
p = People()

In [5]:
p.birth

__get__:
	 self:<__main__.BirthdayField object at 0x1056e0650>
	 obj:<__main__.People object at 0x10688fb10>
	 Object Type:<class '__main__.People'>


'2000年1月1日'

In [6]:
p.birth = "new value"

In [7]:
p.birth

'new value'

**发现当我们给`p.birth`重新复制之后，获取值的时候已经不再进入`__get__`方法了**

### 3. 数据描述符
> `data descriptor`，就是同时实现了`__get__`和`__set__`方法的描述符。

In [8]:
class BirthdayField(object):
    
    def __init__(self, value):
        self.value = value
    
    def __get__(self, obj, objtype=None):
        # 这里obj会是引用到BirthdayField的类
        # objtype是引用这个属性描述符的类
        print("__get__:")
        print("\t self:{}".format(self))
        print("\t obj:{}".format(obj))
        print("\t Object Type:{}".format(objtype))
        return self.value
    
    def __set__(self, obj, value):
        print("___set__:")
        print("__get__:")
        print("\t self:{}".format(self))
        print("\t obj:{}".format(obj))
        print("\t Value:{}".format(value))
        if value != None:
            self.value = value
        else:
            raise ValueError("请传入value")
            
    def __delete__(self, obj):
        pass

In [9]:
class People:
    
    birth = BirthdayField("2000年1月1日")

In [10]:
People.birth

__get__:
	 self:<__main__.BirthdayField object at 0x10688fbd0>
	 obj:None
	 Object Type:<class '__main__.People'>


'2000年1月1日'

In [11]:
# 实例化对象
p = People()

In [12]:
# 获取对象的属性
p.birth

__get__:
	 self:<__main__.BirthdayField object at 0x10688fbd0>
	 obj:<__main__.People object at 0x10689d990>
	 Object Type:<class '__main__.People'>


'2000年1月1日'

In [13]:
# 给属性赋值
p.birth = "2000年9月1日"

___set__:
__get__:
	 self:<__main__.BirthdayField object at 0x10688fbd0>
	 obj:<__main__.People object at 0x10689d990>
	 Value:2000年9月1日


In [14]:
p.birth

__get__:
	 self:<__main__.BirthdayField object at 0x10688fbd0>
	 obj:<__main__.People object at 0x10689d990>
	 Object Type:<class '__main__.People'>


'2000年9月1日'

**发现给`p.birth`重新复制之后，获取值的时候，依然进入`__get__`方法**。而`non-data descriptor`的就不会。

### 4. 示例
> 创建个AgeField的类，让其值是0-200之间。然后User使用AgeField。

In [15]:
import numbers

In [16]:
# 创建数据描述符
class AgeField:
    
    def __init__(self, value=0):
        self.__value = value
        
    def __get__(self, obj, objtype=None):
        return self.__value
    
    def __set__(self, obj, value):
        # 传递的值需要是整型
        if isinstance(value, numbers.Integral):
            if value >=0 and value <= 200:
                self.__value = value
            else:
                raise ValueError("值需要是0-200之间")
        else:
            raise ValueError("需要传递整数类型")

In [17]:
# 创建User类
class User:
    age = AgeField()

In [18]:
# 实例化User
user = User()

In [19]:
print(user.age)

0


In [20]:
user.age = 20

In [21]:
user.age

20

In [22]:
# 测试超出范围的值
user.age = 999

ValueError: 值需要是0-200之间

In [23]:
# 设置错误类型的值
user.age = "abc"

ValueError: 需要传递整数类型

**AgeField**当设置的值不正确时会抛出异常。  
熟悉了属性描述符的基本使用，再去看`Django ORM`的相关源码就非常容易理解了。